In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('temp_data2.csv')
df['latitude'] = df['latitude'].apply(lambda x: x[:-1])
df['longitude'] = df['longitude'].apply(lambda x: x[:-1])
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].astype(float)
date_time = pd.to_datetime(df.pop('time'), format='%Y%m%d%H')
# df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H')
df.head()

,id,latitude,longitude,speed
0,0,128.0,904.0,60
1,0,134.0,906.0,75
2,0,140.0,907.0,85
3,0,145.0,908.0,65
4,0,150.0,909.0,60


In [3]:
cyc_id_ds = df.id.unique()
np.random.shuffle(cyc_id_ds)
# print(cyc_id_ds)
n = len(cyc_id_ds)
train_ids = cyc_id_ds[0:int(n*0.7)]
val_ids = cyc_id_ds[int(n*0.7):int(n*0.9)]
test_ids = cyc_id_ds[int(n*0.9):]
# print(train_ids)

train_df = df[0:int(n*0.7)]
# val_df = df[int(n*0.7):int(n*0.9)]
# test_df = df[int(n*0.9):]
# print(type(train_df))

# train_df = df[df.id.isin(train_ids)]
# val_df = df[df.id.isin(val_ids)]
# test_df = df[df.id.isin(test_ids)]

In [4]:
train_df_collection = []
test_df_collection = []
val_df_collection = []


cid = df.pop('id')
MEAN = df.mean()
STD = df.std()
df = (df - MEAN) / STD
df = df.assign(id=cid)


# def normalize(_df):
#     cid = _df.pop('id')

#     mean = _df.mean()
#     std = _df.std()
#     _df = 0.5 * (np.tanh(0.01 * ((_df - mean) / std)) + 1)
#     # _df = (_df - mean) / std
#     _df = _df.assign(id=cid)
#     return _df
# df = normalize(df)

# cid = df.pop('id')
# tanh_mean = df.mean()
# tanh_std = df.std()
# df = 0.5 * (np.tanh(0.01 * ((df - tanh_mean) / tanh_std)) + 1)
# df = df.assign(id=cid)

for _df in [df.loc[df['id'] == id] for id in train_ids]:
    # _df = normalize(_df)
    _df.pop('id')
    train_df_collection.append(_df)

for _df in [df.loc[df['id'] == id] for id in test_ids]:
    # _df = normalize(_df)
    _df.pop('id')
    test_df_collection.append(_df)

for _df in [df.loc[df['id'] == id] for id in val_ids]:
    # _df = normalize(_df)
    _df.pop('id')
    val_df_collection.append(_df)

# count = 0
# for _df in train_df_collection:
#     count += 1
#     print(_df)
# print(count)

for i in val_df_collection:
    print(i)

     latitude  longitude     speed
477 -0.437665  -0.426105 -1.117332
478 -0.375622  -0.458766 -1.117332
479 -0.292899  -0.483261 -1.117332
480 -0.210175  -0.499591 -1.117332
481 -0.127451  -0.524086 -0.894017
482 -0.044728  -0.540416 -0.894017
483  0.037996  -0.564911 -0.894017
484  0.100038  -0.581242 -0.894017
485  0.162081  -0.605737 -0.894017
486  0.306847  -0.622067 -0.670702
487  0.472294  -0.646562 -0.670702
488  0.534337  -0.662892 -0.670702
489  0.596380  -0.679222 -0.670702
490  0.720465  -0.720048 -0.670702
491  0.844550  -0.769038 -0.447388
492  0.906593  -0.826194 -0.224073
493  0.947955  -0.891514 -0.000758
494  1.030678  -0.973165  0.222557
495  1.092721  -1.062981  0.222557
496  1.113402  -1.144631  0.445871
497  1.134083  -1.218117  0.669186
498  1.154763  -1.316098  0.445871
499  1.154763  -1.405914  0.222557
500  1.154763  -1.454904 -0.000758
501  1.154763  -1.512060 -0.224073
502  1.154763  -1.561050 -0.447388
503  1.134083  -1.618206 -0.447388
504  1.154763  -1.67

In [5]:
print(MEAN)
print(STD)

latitude     140.162766
longitude    784.186350
speed         40.016973
dtype: float64
latitude      48.353828
longitude    122.472912
speed         22.389924
dtype: float64


In [5]:

class WindowGenerator:
    def __init__(
        self,
        input_width,
        label_width,
        shift,
        train_df,
        val_df,
        test_df,
        label_columns=None,
    ):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {
                name: i for i, name in enumerate(label_columns)
            }

        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return "\n".join(
            [
                f"Total window size: {self.total_window_size}",
                f"Input indices: {self.input_indices}",
                f"Label indices: {self.label_indices}",
                f"Label column name(s): {self.label_columns}",
            ]
        )

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]

        if self.label_columns is not None:
            labels = tf.stack(
                [
                    labels[:, :, self.column_indices[name]]
                    for name in self.label_columns
                ],
                axis=-1,
            )

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            # shuffle=True,
            # batch_size=32,
        )

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, "_example", None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result
    
def plot(train_ds, window, model=None, plot_col='speed', max_subplots=3):
    inputs, labels = next(iter(train_ds))
    plt.figure(figsize=(12, 8))

    column_indices = {name: i for i, name in enumerate(df.columns)}
    plot_col_index = column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))

    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(window.input_indices, inputs[n, :, plot_col_index],
                label='Inputs', marker='.', zorder=-10)

        if window.label_columns:
            label_col_index = window.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(window.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        
        if model is not None:
            predictions = model(inputs)
            plt.scatter(window.label_indices, predictions[n, :, label_col_index],
                    marker='X', edgecolors='k', label='Predictions',
                    c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [h]')


In [6]:
def create_tensor_timeseries_dataset(input_width, label_width, shift):
    train_ds = None
    for tdf in train_df_collection:
        w = WindowGenerator(input_width=input_width, label_width=label_width, shift=shift, train_df=tdf, val_df=None, test_df=None, label_columns=['speed'])
        if train_ds == None:
            train_ds = w.train
        else:
            train_ds = train_ds.concatenate(w.train)    

    val_ds = None
    for tdf in val_df_collection:
        w = WindowGenerator(input_width=input_width, label_width=label_width, shift=shift, train_df=tdf, val_df=None, test_df=None, label_columns=['speed'])
        if val_ds == None:
            val_ds = w.train
        else:
            val_ds = val_ds.concatenate(w.train)

    test_ds = None
    for tdf in test_df_collection:
        w = WindowGenerator(input_width=input_width, label_width=label_width, shift=shift, train_df=tdf, val_df=None, test_df=None, label_columns=['speed'])
        if test_ds == None:
            test_ds = w.train
        else:
            test_ds = test_ds.concatenate(w.train)
    
    return train_ds, val_ds, test_ds

In [7]:
train_ds, val_ds, test_ds = create_tensor_timeseries_dataset(4, 1, 1)

In [8]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [9]:
baseline = Baseline(label_index=2)

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError(), 'accuracy'])

val_performance = {}
performance = {}
prediction_error = {}
val_performance['Baseline'] = baseline.evaluate(val_ds)
performance['Baseline'] = baseline.evaluate(test_ds, verbose=0)

50/50 [==============================] - 4s 29ms/step - loss: 0.4252 - mean_absolute_error: 0.4261 - accuracy: 0.0000e+00


In [10]:
MAX_EPOCHS = 10


def compile_and_fit(model, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=patience, mode="min"
    )

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.RootMeanSquaredError(),
        ],
    )

    history = model.fit(
        train_ds, epochs=MAX_EPOCHS, validation_data=val_ds, callbacks=[early_stopping]
    )
    return history


def get_prediction_error(model, test_ds):
    count = 0
    diff = 0
    for i in test_ds.as_numpy_iterator():
        input, label = i
        prediction = np.array(model(input))

        for i, p in enumerate(prediction):
            p = p * STD["speed"] + MEAN["speed"]
            a = label[i] * STD["speed"] + MEAN["speed"]
            diff += np.abs(a - p)
            print(a.shape, p.shape, diff.shape)
            count += 1
    
    return diff/count


In [11]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=4),
    tf.keras.layers.Dense(units=1)
])

history = compile_and_fit(linear)

val_performance['Linear'] = linear.evaluate(val_ds)
performance['Linear'] = linear.evaluate(test_ds, verbose=0)
prediction_error['Linear'] = get_prediction_error(linear, test_ds)

Epoch 1/10
174/174 [==============================] - 5s 23ms/step - loss: 1.0028 - mean_absolute_error: 0.7427 - root_mean_squared_error: 1.0014 - val_loss: 1.3033 - val_mean_absolute_error: 0.7777 - val_root_mean_squared_error: 1.1416
Epoch 2/10
174/174 [==============================] - 4s 25ms/step - loss: 0.8193 - mean_absolute_error: 0.6689 - root_mean_squared_error: 0.9052 - val_loss: 1.1131 - val_mean_absolute_error: 0.7183 - val_root_mean_squared_error: 1.0550
Epoch 3/10
174/174 [==============================] - 5s 29ms/step - loss: 0.7034 - mean_absolute_error: 0.6206 - root_mean_squared_error: 0.8387 - val_loss: 0.9441 - val_mean_absolute_error: 0.6636 - val_root_mean_squared_error: 0.9717
Epoch 4/10
174/174 [==============================] - 8s 43ms/step - loss: 0.5914 - mean_absolute_error: 0.5704 - root_mean_squared_error: 0.7691 - val_loss: 0.7609 - val_mean_absolute_error: 0.5993 - val_root_mean_squared_error: 0.8723
Epoch 5/10
174/174 [==============================] 

In [12]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

history = compile_and_fit(dense)

val_performance['Dense'] = dense.evaluate(val_ds)
performance['Dense'] = dense.evaluate(test_ds, verbose=0)
prediction_error['Dense'] = get_prediction_error(dense, test_ds)

Epoch 1/10
174/174 [==============================] - 9s 44ms/step - loss: 0.4089 - mean_absolute_error: 0.4587 - root_mean_squared_error: 0.6394 - val_loss: 0.3782 - val_mean_absolute_error: 0.4223 - val_root_mean_squared_error: 0.6150
Epoch 2/10
174/174 [==============================] - 9s 49ms/step - loss: 0.2773 - mean_absolute_error: 0.3595 - root_mean_squared_error: 0.5265 - val_loss: 0.3674 - val_mean_absolute_error: 0.4160 - val_root_mean_squared_error: 0.6061
Epoch 3/10
174/174 [==============================] - 8s 47ms/step - loss: 0.2731 - mean_absolute_error: 0.3566 - root_mean_squared_error: 0.5225 - val_loss: 0.3624 - val_mean_absolute_error: 0.4130 - val_root_mean_squared_error: 0.6020
Epoch 4/10
174/174 [==============================] - 8s 47ms/step - loss: 0.2702 - mean_absolute_error: 0.3551 - root_mean_squared_error: 0.5198 - val_loss: 0.3591 - val_mean_absolute_error: 0.4104 - val_root_mean_squared_error: 0.5992
Epoch 5/10
174/174 [==============================] 

In [13]:
CONV_WIDTH = 4
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

history = compile_and_fit(conv_model)

val_performance['Conv'] = conv_model.evaluate(val_ds)
performance['Conv'] = conv_model.evaluate(test_ds, verbose=0)
prediction_error['Conv'] = get_prediction_error(linear, test_ds)

Epoch 1/10
174/174 [==============================] - 7s 37ms/step - loss: 0.2279 - mean_absolute_error: 0.3260 - root_mean_squared_error: 0.4773 - val_loss: 0.1128 - val_mean_absolute_error: 0.2383 - val_root_mean_squared_error: 0.3359
Epoch 2/10
174/174 [==============================] - 7s 38ms/step - loss: 0.0755 - mean_absolute_error: 0.1927 - root_mean_squared_error: 0.2749 - val_loss: 0.0674 - val_mean_absolute_error: 0.1873 - val_root_mean_squared_error: 0.2595
Epoch 3/10
174/174 [==============================] - 8s 46ms/step - loss: 0.0607 - mean_absolute_error: 0.1717 - root_mean_squared_error: 0.2464 - val_loss: 0.0596 - val_mean_absolute_error: 0.1768 - val_root_mean_squared_error: 0.2442
Epoch 4/10
174/174 [==============================] - 6s 33ms/step - loss: 0.0558 - mean_absolute_error: 0.1654 - root_mean_squared_error: 0.2362 - val_loss: 0.0563 - val_mean_absolute_error: 0.1718 - val_root_mean_squared_error: 0.2372
Epoch 5/10
174/174 [==============================] 

In [14]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(10, recurrent_activation='tanh', return_sequences=False),
    tf.keras.layers.Dense(units=1, activation='tanh'),
])

history = compile_and_fit(lstm_model)

val_performance['LSTM'] = lstm_model.evaluate(val_ds)
performance['LSTM'] = lstm_model.evaluate(test_ds, verbose=0)
prediction_error['LSTM'] = get_prediction_error(lstm_model, test_ds)

Epoch 1/10
174/174 [==============================] - 7s 29ms/step - loss: 0.7105 - mean_absolute_error: 0.6253 - root_mean_squared_error: 0.8429 - val_loss: 0.9016 - val_mean_absolute_error: 0.6841 - val_root_mean_squared_error: 0.9495
Epoch 2/10
174/174 [==============================] - 4s 22ms/step - loss: 0.5001 - mean_absolute_error: 0.5250 - root_mean_squared_error: 0.7072 - val_loss: 0.7434 - val_mean_absolute_error: 0.6047 - val_root_mean_squared_error: 0.8622
Epoch 3/10
174/174 [==============================] - 4s 23ms/step - loss: 0.4031 - mean_absolute_error: 0.4426 - root_mean_squared_error: 0.6349 - val_loss: 0.6656 - val_mean_absolute_error: 0.5400 - val_root_mean_squared_error: 0.8158
Epoch 4/10
174/174 [==============================] - 4s 23ms/step - loss: 0.3576 - mean_absolute_error: 0.3920 - root_mean_squared_error: 0.5980 - val_loss: 0.6250 - val_mean_absolute_error: 0.4971 - val_root_mean_squared_error: 0.7906
Epoch 5/10
174/174 [==============================] 

In [15]:
# train_ds, val_ds, test_ds = create_tensor_timeseries_dataset(4, 1, 4)

In [16]:
# linear = tf.keras.Sequential([
#     tf.keras.layers.Dense(units=4),
#     tf.keras.layers.Dense(units=1)
# ])

# history = compile_and_fit(linear)

# val_performance['Linear'] = linear.evaluate(val_ds)
# performance['Linear'] = linear.evaluate(test_ds, verbose=0)
# prediction_error['Linear'] = get_prediction_error(linear)

In [17]:
# print(get_prediction_error(linear, test_ds))

In [18]:
import pickle
with open('cyclone_intensity_prediction.pickle','wb') as f:
    pickle.dump(lstm_model,f)